# SIC Capstone project

## Class hierarchy

### Agent

In [4]:
class Agent:
    def __init__(self, name):
        self.name = name

    def describe(self):
        return f'{self.name}'

### Client

In [6]:
class Client(Agent):
    def __init__(self, name):
        super().__init__(name)

### Theater


In [8]:
# TICKET CLASS 
from collections import deque # https://docs.python.org/3/library/collections.html#collections.deque

class TicketQueue:
    def __init__(self):
        self.queue = deque() 

    def enqueue(self, client_name):
        self.queue.append(client_name)
        print(f'Client {client_name} added to the ticket queue.')

    def dequeue(self):
        if self.queue:
            client_name = self.queue.popleft()
            print(f'Client {client_name} removed from the ticket queue.')
            return client_name
        else:
            print('The ticket queue is empty.')
            return None
            
    def show_queue(self):
        if self.is_empty():
            print('The ticket queue is empty.')
        else:
            print('Current ticket queue:')
            for client in self.queue:
                print(f'- {client}')

    def remove_client(self, client_name):
        try:
            self.queue.remove(client_name)
            print(f'Client {client_name} removed from the ticket queue.')
        except ValueError:
            print(f'Client {client_name} not found in the queue.')
    
    def is_empty(self):
        return len(self.queue) == 0

    def front(self): 
        if self.queue:
            return self.queue[0]
        return None

    def get_all_clients(self):
        return list(self.queue)  
        
#THEATER CLASS
class Theater(Agent):   
    def __init__(self, name):
        super().__init__(name)
        self.shows = {}
        self.ticket_queue = TicketQueue()  
        self.reservations = {}  
        self.visitors = {}  

    def add_show(self, show_name, details=""):
        self.shows[show_name] = details
        print(f"Show '{show_name}' added to {self.name}.")

    def remove_show(self, show_name):
        if show_name in self.shows:
            if self.shows[show_name]['visitors']:
                print(f"Cannot remove show '{show_name}'. There are clients watching it.")
            else:
                del self.shows[show_name]
                print(f"Show '{show_name}' removed from {self.name}.")
        else:
            print(f"Show '{show_name}' does not exist in {self.name}.")

    def leave_show(self, client_name, show_name):
        if show_name in self.shows and client_name in self.shows[show_name]['visitors']:
            self.shows[show_name]['visitors'].remove(client_name)
            print(f'Client {client_name} left the show "{show_name}" in theater {self.name}.')
        else:
            print(f'Client {client_name} is not viewing show "{show_name}" in theater {self.name}.')
    
    def reserve_ticket(self, client_name, show_name):
        if show_name in self.shows:
            if self.ticket_queue.get_all_clients():  
                if client_name in self.ticket_queue.get_all_clients():
                    print(f"Client {client_name} already has a reservation for show '{show_name}'.")
                    return  
            self.ticket_queue.enqueue(client_name)  
            if show_name not in self.reservations:
                self.reservations[show_name] = []
            self.reservations[show_name].append(client_name) 
            print(f'Client {client_name} reserved a ticket for show "{show_name}".')
        else:
            print(f'Show {show_name} not found in {self.name}.')
            
    def cancel_reservation(self, client_name, show_name):
        if show_name in self.shows:
            if client_name in self.ticket_queue.get_all_clients():
                self.ticket_queue.remove_client(client_name)  
                if show_name not in self.visitors:
                    print(f"No reservations found for client {client_name} for show '{show_name}'.")
                    return
                if client_name in self.visitors[show_name]:
                    self.visitors[show_name].remove(client_name) 
                    print(f"Reservation for client {client_name} for show '{show_name}' cancelled.")
                else:
                    print(f"No reservation found for client {client_name} for show '{show_name}'.")
            else:
                print(f"Client {client_name} not found in ticket queue.")
        else:
            print(f"Show '{show_name}' not found in theater {self.name}.")
    
    def show_ticket_queue(self):
        self.ticket_queue.show_queue()

    
    def has_revervation(self, client_name):
        return client_name in self.ticket_queue.get_all_clients()
   
    def buy_ticket(self, client_name):
        if not self.ticket_queue.is_empty() and self.ticket_queue.front() == client_name:
            self.ticket_queue.dequeue()
            print(f'Client {client_name} bought a ticket.')
            
            for show_name in self.reservations.keys():
                if client_name in self.reservations[show_name]:
                    if show_name not in self.visitors:
                        self.visitors[show_name] = []
                    self.visitors[show_name].append(client_name)
                    print(f'Client {client_name} is now viewing the show "{show_name}".')
            return
        else:
            print(f'Client {client_name} is not at the front of the queue. Cannot buy ticket.')

    def go_to_show(self, client_name, show_name):
        if show_name not in self.shows:
            print(f'Show "{show_name}" not found in {self.name}.')
            return

        if show_name in self.reservations and client_name in self.reservations[show_name]:
            if client_name not in self.visitors.get(show_name, []):
                print(f'Client {client_name} entered the show "{show_name}" in {self.name}.')
                self.visitors.setdefault(show_name, []).append(client_name) 
                if client_name in self.ticket_queue.get_all_clients():
                    self.ticket_queue.dequeue() 
            else:
                print(f'Client {client_name} is already in the show "{show_name}".')
        else:
            print(f'Client {client_name} cannot enter the show "{show_name}". Please ensure you have a ticket.')

### City simulation

In [10]:
import time

class CitySimulation:
    def __init__(self):       
        self.agents = {}

    def show_theaters(self):
        if not self.agents:
            print("No theaters available.")
            return
        print("Available theaters and their shows:")
        for theater_name, theater in self.agents.items():
            if isinstance(theater, Theater):
                print(f"{theater_name}: {list(theater.shows.keys())}")  
    
    def add_agent(self, agent_type, agent_name):    
        if agent_type == 'client':
            self.agents[agent_name] = Client(agent_name)
            print(f'Client {agent_name} added.')           
        elif agent_type == 'town_hall':
            self.agents[agent_name] = TownHall(agent_name)
            print(f'Town Hall {agent_name} added.')            
        elif agent_type == 'theater':
            self.agents[agent_name] = Theater(agent_name)
            print(f'Theater "{agent_name}" added.')           
        else:
            print(f"Unknown agent type: {agent_type}")
        
    def remove_agent(self, agent_name):
        if agent_name in self.agents:
            del self.agents[agent_name]
            print(f'Agent {agent_name} removed from the system.')
        else:
            print(f'Agent {agent_name} not found.')

    def remove_theater(self, theater_name):
        if theater_name in self.agents and isinstance(self.agents[theater_name], Theater):
            del self.agents[theater_name]
            print(f'Theater "{theater_name}" removed from the system.')
        else:
            print(f'Theater "{theater_name}" not found or is not a theater.')

    def list_agents(self):
        print("Current agents:")
        for agent in agents.values():
            print(agent.describe())

    def help(self):
        print('''Available commands:
            - For Clients/Theaters commands type "help_theater"
            - For Clients/Shows commands type "help_shows"
            - For exit program type 'q'
            ''')
        

    def help_theater(self):
        print("""
            Available commands for clients/theater help:
            - theater add <theater name> : for creating a new Theater         
            - theater show_all_theater : a list of theaters on the system
            - client enter <client_name> <theater_name> : enter to a Theater
            - client leave <client_name> <theater_name> : leave the Theater
            """)

    def help_shows(self):
        print("""
            Available commands for client/show help:
            - theater add_show <theater_name> <show_name> : for creating a new show in the theater
            - client go_to_show <client_name> <show_name> : go to a show    
            - client make_revervation <client_name> <theater_name> <show_name> : made a reservation for a show
            - client buy_ticket <client_name> <theater_name> <show_name> : buy the ticket after the reservation and see the show
            - client cancel_revervation <client_name> <theater_name> <show_name> : cancel reservation for a show
            - theater show_queue <theater_name> : to see the queue in the theater
            """)
    
    def command_loop(self):
        time.sleep(0.5)
        print('Starting program...')
        time.sleep(1)
        self.help()
        while True:
            command = input('> ')
            if command == 'q':
                docs = '...'
                for i in docs:
                    print(i)
                    time.sleep(0.5)
                print("Ending program :)")
                break
            self.process_command(command)

    def process_command(self, command):
        parts = command.split()
        if not parts:
            return
        
        cmd = parts[0]      
        
        if cmd == 'help':
            pass     
        if cmd == 'help_theater':
            self.help_theater()  
            return          
        if cmd == 'help_shows':
            self.help_shows() 
            return        
                    
        if cmd == 'client':
            if len(parts) <= 2:
                print("Error: Incomplete command for client.")
                return
            if parts[1] == 'add':
                client_name = parts[2]
                if client_name not in self.agents:
                    self.agents[client_name] = Client(client_name)
                    print(f'Client {client_name} added.')
                else:
                    print(f'Client {client_name} already exists.')
            
            if parts[1] == 'leave_show':
                try:
                    client_name = parts[2]
                    theater_name = parts[3]
                    show_name = parts[4]
                    if client_name in self.agents and isinstance(self.agents[client_name], Client):
                        if theater_name in self.agents and isinstance(self.agents[theater_name], Theater):
                            self.agents[theater_name].leave_show(client_name, show_name)
                        else:
                            print(f'Theater {theater_name} not found.')
                    else:
                        print(f'Client {client_name} not found.')
                except IndexError:
                    print("Error: Invalid leave_show command format. Use 'client leave_show <client_name> <theater_name> <show_name>'")
                    
            elif parts[1] == 'make_reservation':
                try:
                    client_name = parts[2]  
                    theater_name = parts[3]  
                    show_name = parts[4] 

                    if client_name not in self.agents or not isinstance(self.agents[client_name], Client):
                        print(f'Client {client_name} not found.')
                        return

                    if theater_name not in self.agents or not isinstance(self.agents[theater_name], Theater):
                        print(f'Theater {theater_name} not found.') 
                        return

                    theater = self.agents[theater_name]
        
                    if show_name not in theater.shows:
                        print(f'Show {show_name} not found in theater {theater_name}.')
                        return 
                                                 
                    theater.reserve_ticket(client_name, show_name)

                except IndexError:
                    print("Error: Invalid make_reservation command format. Use 'client make_reservation <client_name> <theater_name> <show_name>'")  # Mensaje de error
                    
            elif parts[1] == 'cancel_reservation':
                try:
                    client_name = parts[2]
                    theater_name = parts[3]
                    show_name = parts[4]

                    if client_name not in self.agents or not isinstance(self.agents[client_name], Client):
                        print(f'Client {client_name} not found.')
                        return

                    if theater_name not in self.agents or not isinstance(self.agents[theater_name], Theater):
                        print(f'Theater {theater_name} not found.')
                        return

                    theater = self.agents[theater_name]
                    theater.cancel_reservation(client_name, show_name)

                except IndexError:
                    print("Error: Invalid cancel_reservation command format. Use 'client cancel_reservation <client_name> <theater_name> <show_name>'")

            elif parts[1] == 'buy_ticket':   
                try:
                    client_name = parts[2]  
                    theater_name = parts[3] 
                    
                    if client_name not in self.agents or not isinstance(self.agents[client_name], Client):
                        print(f'Client {client_name} not found.')
                        return
                    if theater_name not in self.agents or not isinstance(self.agents[theater_name], Theater):
                        print(f'Theater {theater_name} not found.')
                        return

                    self.agents[theater_name].buy_ticket(client_name)

                except IndexError:
                    print("Error: Invalid buy_ticket command format. Use 'client buy_ticket <client_name> <theater_name> <show_name>'")
                        
            elif parts[1] == 'enter': 
                try:
                    client_name = parts[2]
                    theater_name = parts[3]
                    if client_name in self.agents and isinstance(self.agents[client_name], Client):
                        if theater_name in self.agents and isinstance(self.agents[theater_name], Theater):
                            print(f'Client {client_name} entered the theater {theater_name}.')
                        else:
                            print(f'Theater {theater_name} not found.')
                    else:
                        print(f'Client {client_name} not found.')
                except IndexError:
                    print("Error: Invalid enter command format. Use 'client enter <client_name> <theater_name>'") 
                              
            elif parts[1] == 'go_to_show':
                try:
                    client_name = parts[2]  
                    theater_name = parts[3]  
                    show_name = parts[4]  

                    
                    if client_name not in self.agents or not isinstance(self.agents[client_name], Client):
                        print(f'Client {client_name} not found.')
                        return

                    if theater_name not in self.agents or not isinstance(self.agents[theater_name], Theater):
                        print(f'Theater {theater_name} not found.') 
                        return

                    theater = self.agents[theater_name]
        
                    if show_name not in theater.shows:
                        print(f'Show "{show_name}" not found in theater {theater_name}.')
                        return 
                     
                    if (client_name in theater.ticket_queue.get_all_clients() or
                        (show_name in theater.visitors and client_name in theater.visitors[show_name])):
                            theater.go_to_show(client_name, show_name)
                        
                    else:
                        print(f'Client {client_name} does not have a ticket for show "{show_name}".')

                except IndexError:
                    print("Error: Invalid go_to_show command format. Use 'client go_to_show <client_name> <theater_name> <show_name>'")  # Mensaje de error

            elif parts[1] == 'leave': 
                try:
                    client_name = parts[2]
                    theater_name = parts[3]
                    if client_name in self.agents and isinstance(self.agents[client_name], Client):
                        print(f'Client {client_name} left the theater {theater_name}')
                    else:
                        print(f'Client {client_name} not found.')
                except IndexError:
                    print("Error: Invalid enter command format. Use 'client leave <client_name> <theater_name>'")

            elif parts[1] == 'leave_show':
                try:
                    client_name = parts[2]
                    theater_name = parts[3]
                    show_name = parts[4]
                    if client_name in self.agents and isinstance(self.agents[client_name], Client):
                        print(f'Client {client_name} left the show {show_name} in theater {theater_name}.')
                    else:
                        print(f'Client {client_name} not found')
                except IndexError:
                    print("Error: Invalid leave_show command format. Use 'client leave_show <client_name> <theater_name> <show_name>'")      
            
        if cmd == 'theater':
            if parts[1] == 'add':
                theater_name = parts[2]
                if theater_name not in self.agents:
                    self.add_agent('theater', theater_name)
                else:
                    print(f'Theater {theater_name} already exists.')

            if parts[1] == 'remove_theater':
                try:
                    theater_name = parts[2]
                    self.remove_theater(theater_name)
                except IndexError:
                    print("Error: Invalid remove_theater command format. Use 'theater remove_theater <theater_name>'")
                    
            elif parts[1] == 'show_queue':
                try:
                    theater_name = parts[2]  
                    if theater_name in self.agents and isinstance(self.agents[theater_name], Theater):
                        self.agents[theater_name].show_ticket_queue() 
                    else:
                        print(f'Theater {theater_name} not found.')
                except IndexError:
                    print("Error: Invalid show_queue command format. Use 'theater show_queue <theater_name>'")

            elif parts[1] == 'show_all_theater':
                theaters = [agent for agent in self.agents.values() if isinstance(agent, Theater)]
                if not theaters:
                    print('No theaters available.')
                else:
                    print('Current theaters:')
                    for name, agent in self.agents.items():
                        if isinstance(agent, Theater):
                            print(f"- {name}")
                            if agent.shows:
                                for show in agent.shows:
                                    print(f"   * Show: {show}")
                            else:
                                print("No shows available.")
                    
            elif parts[1] == 'add_show':
                try:
                    theater_name = parts[2]
                    show_name = parts[3]  
                    if theater_name in self.agents and isinstance(self.agents[theater_name], Theater):
                        self.agents[theater_name].add_show(show_name)
                    else:
                        print(f'Theater {theater_name} not found.')
                except IndexError:
                    print("Error: Invalid add_show command format. Use 'theater add_show <theater_name> <show_name>'")

## Main program

In [12]:
if __name__ == "__main__":
    simulation = CitySimulation()
    simulation.command_loop()

Starting program...
Available commands:
            - For Clients/Theaters commands type "help_theater"
            - For Clients/Shows commands type "help_shows"
            - For exit program type 'q'
            


>  client add Michael


Client Michael added.


>  client add Jesus


Client Jesus added.


...

Theater "Teatro_Real" removed from the system.


>  theater show_all_theater


Current theaters:
- Teatro_de_Madrid
   * Show: My_show
   * Show: Musical


>  q


.
.
.
Ending program :)
